In [6]:
import tensorflow as tf
from keras.models import load_model
from keras.layers import *
from keras import Model, Sequential, Input
from keras import optimizers
from keras.regularizers import l2
import keras.backend as K
import numpy as np
from pprint import pprint
import pickle


def load(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

dataTriple = load('obj/triplets.pkl')
triplets = dataTriple['triplets'] / 256

In [7]:
def get_darknet():
    model = load_model('darknet.h5')
    for layer in model.layers:
        layer.trainable = False
    return model

def make_style_model():
    darknet = get_darknet()
    
    # pretrained layers
    inputs = darknet.layers[0].input
    x = darknet.layers[16].output
    
    # new layers
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Lambda(lambda x: tf.nn.l2_normalize(x, axis=-1))(x)
    # Embedding model produces style embeddings
    embedding_model = Model(inputs, x)
    
    # Crate triple model to train the base model
    # inputs for anchor, positive image, and negative image
    input_anc = Input((256,256,3))
    input_pos = Input((256,256,3))
    input_neg = Input((256,256,3))
    
    # the embedding of the anchor, positive image, and negative image
    embed_anc = embedding_model(input_anc)
    embed_pos = embedding_model(input_pos)
    embed_neg = embedding_model(input_neg)
    
    # distance between anchor and positive embeddings
    dist_pos = Lambda(sqEucl, output_shape=(1,))([embed_anc, embed_pos])
    # distance between anchor and negative embeddings
    dist_neg = Lambda(sqEucl, output_shape=(1,))([embed_anc, embed_neg])
    
    # The tiplet_loss: dist_pos - dist_neg + 1 < 0
    triple_loss = Lambda(triplet_loss, output_shape=(1,))([dist_pos, dist_neg])
    # triplet model, for training the embedding model
    triple_model = Model([input_anc, input_pos, input_neg], triple_loss)
    
    return triple_model, embedding_model

def triplet_loss(x):
    return K.maximum(x[0] - x[1] + 1, 0)

def sqEucl(x):
    return K.sum(K.square(x[0] - x[1]), axis=-1, keepdims=True)

def norm(x):
    return K.sqrt(sqEucl(x))
    
def identity(y_true, y_pred):
    return K.mean(K.square(y_pred))

In [ ]:
tripleModel, pairModel = make_style_model()

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
tripleModel.compile(optimizer=sgd, loss=identity)
pairModel.compile(optimizer=sgd, loss=identity)
alphas = np.zeros(triplets.shape[0])
tripleModel.fit([triplets[:, 0], triplets[:, 1], triplets[:, 2]], alphas, epochs=50)

embedding_model.save('obj/style_emb_model.h5')

/home/henrik/miniconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/50
  64/1000 [>.............................] - ETA: 3:59 - loss: 1.0882